## Parse Messenger inbox

In [ ]:
from datetime import datetime
import glob
import json

In [ ]:
chats = glob.glob("facebook-jasoncbenn/messages/inbox/*/*.json")

In [ ]:
from datetime import timedelta
import unidecode
from collections import defaultdict

MY_NAME = 'Jason Benn'

interactions = defaultdict(lambda: {'interaction_days': set()})

def parse_datetime(dt: int) -> datetime:
    return datetime.fromtimestamp(dt / 1e3)

for chat_json_path in chats:
    with open(chat_json_path, encoding='raw_unicode_escape') as f:
        chat = json.loads(f.read().encode('raw_unicode_escape').decode())

    others = [x['name'] for x in chat['participants'] if x['name'] != MY_NAME]

    my_messages = [x for x in chat['messages'] if x['sender_name'] == MY_NAME]

    if not len(my_messages):
        continue 

    last_message_time = None

    is_groupchat = len(others) > 1

    if is_groupchat:
        # Creative technique: anytime I message the group, I get "credit" for all messages within 2 days
        # Goal is to account for groups where I'm active + others are active - those feel like touches
        # But many other groups are too big, too sparse, not mine, etc. Those don't count.
        for message in reversed(chat['messages']):
            msg_time = parse_datetime(message['timestamp_ms'])
            msg_name = unidecode.unidecode(message['sender_name'])
            if msg_name == MY_NAME:
                last_message_time = msg_time
                continue

            if last_message_time and last_message_time + timedelta(days=2) > msg_time:
                interactions[msg_name]['interaction_days'].add(msg_time.date())

    else:
        # If this is 1:1, just count every unique day of any message.
        for message in chat['messages']:
            msg_time = parse_datetime(message['timestamp_ms'])
            msg_name = unidecode.unidecode(message['sender_name'])
            if msg_name == MY_NAME:
                continue
            interactions[msg_name]['interaction_days'].add(msg_time.date())

In [ ]:
compressed_interactions = {}
for name, touches in interactions.items():
    compressed_interactions[name] = {'last_interaction': max(touches['interaction_days']), 'num_interaction_days': len(touches['interaction_days'])}

sorted(compressed_interactions.items(), key=lambda x: -x[1]['num_interaction_days'])

## Send to Notion

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
NOTION_SECRET = os.getenv('NOTION_SECRET')
DB_ID = os.getenv('DATABASE_ID')

In [ ]:
import requests

In [ ]:
headers = {'Authorization': f'Bearer {NOTION_SECRET}', 'Notion-Version': '2021-08-16', 'Content-Type': 'application/json'}

def get_table_metadata():
    response = requests.get(f"https://api.notion.com/v1/databases/{DB_ID}", headers=headers)
    return json.loads(response.text)
# sorted(table_metadata['properties'].keys())

In [ ]:
from typing import Optional

def list_db(cursor: Optional[str]):
    data = {
      "sorts": [
        {
          "property": "Name",
          "direction": "ascending"
        }
      ],
      "page_size": 100
    }
    if cursor:
        data['start_cursor'] = cursor
    response = requests.post(f"https://api.notion.com/v1/databases/{DB_ID}/query", headers=headers, data=json.dumps(data))
    response.raise_for_status()
    return json.loads(response.text)

In [ ]:
def date_iso_format(date):
    return datetime.combine(date, datetime.min.time()).isoformat()

In [ ]:
def patch_page_last_connected(page_id, dt):
    data = {'properties': 
        {
         'Last connected': {'date': {'start': date_iso_format(dt), 'end': None}},
        }
       }
    response = requests.patch(f"https://api.notion.com/v1/pages/{page_id}", headers=headers, data=json.dumps(data))
    response.raise_for_status()
    return response

In [ ]:
def patch_page_days_interacted_lately(page_id, days_interacted_lately):
    data = {'properties': 
        {
          'Days interacted lately': days_interacted_lately
        }
       }
    response = requests.patch(f"https://api.notion.com/v1/pages/{page_id}", headers=headers, data=json.dumps(data))
    response.raise_for_status()
    return response

In [ ]:
all_pages = []
pages = list_db(None)
all_pages.extend(pages['results'])

while pages['has_more']:
    print(pages['next_cursor'])
    cursor = pages['next_cursor'] # '8fefddc1-0cb6-4f66-bb07-448073cde34b',
    pages = list_db(cursor)
    all_pages.extend(pages['results'])

print(len(all_pages))

In [ ]:
results_dict = {x['properties']['Name']['title'][0]['text']['content']: x for x in all_pages}

In [ ]:
# for name, interactions in compressed_interactions.items():
#     if name in results_dict:
#         print(f"found {name}")
#     else:
#         print(f"nope  {name}")

In [ ]:
from tqdm import tqdm

for name, interactions in tqdm(compressed_interactions.items()):
    if name in results_dict:
        page_id = results_dict[name]['id']
        patch_page_last_connected(page_id, compressed_interactions[name]['last_interaction'])
        patch_page_days_interacted_lately(page_id, compressed_interactions[name]['num_interaction_days'])